# Example of using QuOCS with Qudi

We want to run an optimization on one of our test-problems provided in QuOCS. TO demonstrate some of the advantages of QuOCS in experiment optimization, we add noise to the figure of merit (FoM) and make use of the re-evaluation steps functionality (see below). But let's go through the setup of the optimization step-py-step.

## Import statements

We need to import the QuOCS Optimizer and the test-problem (OneQubitProblem). Let's also include numpy for good measure becuase we might need it later.

In [1]:
from quocslib.Optimizer import Optimizer
from quocslib.optimalcontrolproblems.OneQubitProblem import OneQubit
import numpy as np
import time

## The Configuration of QuOCS

To configure QuOCS, you need to fill the ```optimization_dictionary``` with information. In this example, we will do it directly in the code, but you can also load the dictionary from a JSON file.

Let's define the name of the optimization. This will also show up in the name of the folder containing the results.

In [2]:
optimization_client_name = "dCRAB_Noisy_2_control_fields"

optimization_dictionary = {"optimization_client_name": optimization_client_name}

Now we need to specify the general settings, such as the used algorithm (in this case dCRAB) and its parameters (e.g. number of super-iterations). In the same section we choos to use the re-evaluation steps method for noisy optimization. Extending the dCRAB algorithm, the direct dearch method (dsm) needs to be specified and stopping criteria for each sub-iteration can be defined.

In [3]:
optimization_dictionary["algorithm_settings"] = {
        "algorithm_name": "dCRAB",
        "super_iteration_number": 5,
        "max_eval_total": 5000,
        "re_evaluation": {
            "re_evaluation_steps": [0.3, 0.5, 0.51]
        },
        "dsm_settings": {
            "general_settings": {
                "dsm_algorithm_name": "NelderMead",
                "is_adaptive": True
            },
            "stopping_criteria": {
                "xatol": 1e-2,
                "fatol": 1e-4,
                "max_eval": 200,
                "change_based_stop": {
                    "cbs_funct_evals": 50,
                    "cbs_change": 0.05
                }
            }
        }
    }

### Times

The time of a pulse is specified separately to use it as an optimization parameter in future updates and so that different pulses can act on separate timescales but can be grouped by common durations. Since there can me more than one time, the entries have to be provided as a list (note the square brackets around the dictionary for ```time_1```):

In [4]:
optimization_dictionary["times"] = [{
        "time_name": "time_1",
        "initial_value": 3.0
    }]

### Parameters

Since we do not have any parameters in this example, we just add an empty list for the settings of parameter optimization.

In [5]:
optimization_dictionary['parameters'] = []

### Pulses

Now we define a pulse to be expanded in the Fourier basis and some other attributes:

In [6]:
pulse_1 = {"pulse_name": "Pulse_1",
           "upper_limit": 15.0,
           "lower_limit": -15.0,
           "bins_number": 101,
           "time_name": "time_1",
           "amplitude_variation": 0.3,
           "basis": {
               "basis_name": "Fourier",
               "basis_vector_number": 2,
               "random_super_parameter_distribution": {
                   "distribution_name": "Uniform",
                   "lower_limit": 0.1,
                   "upper_limit": 5.0
               }
           },
           "scaling_function": {
               "function_type": "lambda_function",
               "lambda_function": "lambda t: 1.0 + 0.0*t"
           },
           "initial_guess": {
               "function_type": "lambda_function",
               "lambda_function": "lambda t: np.pi/3.0 + 0.0*t"
           }
    }

And we instert it into the list of pulses for the settings dict:

In [7]:
optimization_dictionary['pulses'] = [pulse_1]

Finally, we define some parameters for the FoM object:

In [8]:
args_dict = {
        "initial_state": "[1.0 , 0.0]",
        "target_state": "[1.0/np.sqrt(2), -1j/np.sqrt(2)]",
        "is_noisy": True,
        "noise_factor": 0.05
    }

### Now let's put it all together and start the optimization

Load the optimization algorithm into the optimization logic and display it into the GUI

In [9]:
optimizationlogic.load_opti_comm_dict({"optimization_dictionary": optimization_dictionary})

#### Important: If the GUI is not showing the optimization dictionary, restart the Kernel

We have to provide the standard deviation to the dCRAB algorithm. 

To do so we use the function:

``` fomlogic.update_fom(fom, std, status_code=0)```

after the measurement. **std** stands for the experimental standard deviation.

In [10]:
#############################################################################
# To be able to execute and run this example, we use the OneQubit problem   #
# as a placeholder simulation for the real experiment. In a real experiment #
# this definition of the FoM_object is not needed, unless you have written  #
# a class for the execution of your experimental measurement.               #
#############################################################################
# define the FoM object
FoM_object = OneQubit(args_dict=args_dict)

######################################################################################################
# Measurement
######################################################################################################
optimalcontrol.start_optimization()

# Just a time to check for latent time
last_time_fom = time.time()
# repeat the whole process until its manually stopped or QuOCS finsihed the optimization
# Wait few seconds before starting to get and return data
while not optimizationlogic.handle_exit_obj.is_user_running:
    time.sleep(0.1)
    if (time.time() - last_time_fom) > 30:
        print("Problem at the beginning! Surpassed the 30 secs")
        break

# print("Check before the loop starts: {0}".format(optimizationlogic.handle_exit_obj.is_user_running))
while  optimizationlogic.handle_exit_obj.is_user_running == True:
    # Wait until QuOCS optimizes the controls
    # print("Wait until the controls logic gives the controls")
    while not controlslogic.are_pulses_calculated:
        time.sleep(0.1)
        # If the waiting time exceed 10 seconds left stop the optimization
        if time.time() - last_time_fom > 20:
            print("Too much time... Exit!")
            optimizationlogic.handle_exit_obj.is_user_running = False
            break
            

    # Change the status of control calculations to avoid to evaluate the fom twice with the same controls
    controlslogic.are_pulses_calculated = False
    
    # Get the controls from the controls logic
    pulses, parameters, timegrids = controlslogic.pulses, controlslogic.parameters, controlslogic.timegrids
    
    ###########################################################################################
    # Here you can use the pulses, parameters and time grids to call, e.g., your predefined   #
    # method in Qudi and perform your measurement. From the obtained measurement data extract #
    # the FoM and if possible, the standard deviation. if you do not know the standard        #
    # deviation, you can provide a reasonable estimate. Since the standard deviation is only  #
    # used with the re-evaluation steps option, it can simply be set to zero if re-evaluation #
    # is not used.                                                                            #
    ###########################################################################################
    # For our example here, we use the get_FoM function of the OneQubit problem object        #
    ###########################################################################################
    
    # Calculate the Figure of Merit
    fom_dict = FoM_object.get_FoM(pulses, parameters, timegrids)
    
    # Extract the fom and std
    fom, std = fom_dict["FoM"], fom_dict["std"]
    
    ###########################################################################################
    
    # Update the figure of merit and the standard deviation to the FoM logic
    fomlogic.update_fom(fom, std, status_code=0)
    
    # Update the last time the FoM is calculated
    last_time_fom = time.time()
    
    # Print the data just for debug purpose
    print("FoM: {fom}, Std: {std}".format(fom=fom, std=std))
print("Optimization finished")

FoM: 0.6957017068492557, Std: 0.009375856206466911
FoM: 0.6427865787672624, Std: 0.007307614066927547
FoM: 0.7275463432677434, Std: 0.0028450275548001203
FoM: 0.6554372350872756, Std: 0.007493358165270755
FoM: 0.6027991625917037, Std: 0.004488737132974617
FoM: 0.7031507728246754, Std: 0.0016135371578885295
FoM: 0.6216471200946372, Std: 0.008583084933111147
FoM: 0.6407199752055186, Std: 0.0011685906047422302
FoM: 0.5867135689978914, Std: 0.0026922612447858887
FoM: 0.6273829834775346, Std: 0.008362645672851988
FoM: 0.6796649400072933, Std: 0.0033533938596661274
FoM: 0.6144759086254874, Std: 0.007898602913863911
FoM: 0.614577998700609, Std: 0.003090982118012717
FoM: 0.6812012740120238, Std: 0.004993686723131767
FoM: 0.6061884739882355, Std: 0.0049097880315524456
FoM: 0.5458398598263453, Std: 0.003607756238548543
FoM: 0.6283173914697561, Std: 0.0038463646529498974
FoM: 0.6315880326787039, Std: 0.00951183205924651
FoM: 0.5713660935383804, Std: 0.00910644892017022
FoM: 0.6699947907689672, St

FoM: 0.07009422503758644, Std: 0.0031933822500006957
FoM: 0.03362684057974522, Std: 0.001841585987223805
FoM: 0.08632266386105394, Std: 0.00918367529289355
FoM: 0.0175213579385467, Std: 0.005001028184143177
FoM: 0.07135497207094141, Std: 0.0042796774252322365
FoM: 0.007986521172001129, Std: 0.00384558392608559
FoM: 0.012985292285700358, Std: 0.0012184342262924642
FoM: 0.0012485436838636337, Std: 0.0030641295433446072
FoM: 0.011719974750189932, Std: 0.005564983908750573
FoM: 0.036674378026123014, Std: 0.0009582398564089889
FoM: 0.032016972549388366, Std: 0.007447027276252182
FoM: 0.04194588645640094, Std: 0.001317668891451953
FoM: 0.014489225816765532, Std: 0.0012943453096314061
FoM: 0.018784904540356424, Std: 0.0031645943293489263
FoM: 0.057366956535511326, Std: 0.005225497356044736
FoM: 0.034181772450700476, Std: 0.009138267244693134
FoM: 0.050445177473475, Std: 0.00896509187268388
FoM: 0.02465104130111896, Std: 0.008358145337529386
FoM: 0.024868873741010055, Std: 0.001201374914979539

In [11]:
# Access the optimizer object to get info about the optimization
optimization_obj = optimizationlogic.optimization_obj
opt_alg_obj = optimization_obj.get_optimization_algorithm()

In [12]:
# Best FoM with std
print("FoM: {fom} +- {std}".format(fom=opt_alg_obj.best_FoM, std=opt_alg_obj.best_sigma))

In [13]:
# Best controls
best_controls_dict = opt_alg_obj.get_best_controls()
pulse_time = best_controls_dict["timegrids"][0]
pulse_amplitude = best_controls_dict["pulses"][0]
print(pulse_time)
print(pulse_amplitude)

FoM: 0.03253425400257642, Std: 0.0009028185159773228
Optimization finished
FoM: 0.007718998313873579 +- 0.0027362184828384015
[0.   0.03 0.06 0.09 0.12 0.15 0.18 0.21 0.24 0.27 0.3  0.33 0.36 0.39
 0.42 0.45 0.48 0.51 0.54 0.57 0.6  0.63 0.66 0.69 0.72 0.75 0.78 0.81
 0.84 0.87 0.9  0.93 0.96 0.99 1.02 1.05 1.08 1.11 1.14 1.17 1.2  1.23
 1.26 1.29 1.32 1.35 1.38 1.41 1.44 1.47 1.5  1.53 1.56 1.59 1.62 1.65
 1.68 1.71 1.74 1.77 1.8  1.83 1.86 1.89 1.92 1.95 1.98 2.01 2.04 2.07
 2.1  2.13 2.16 2.19 2.22 2.25 2.28 2.31 2.34 2.37 2.4  2.43 2.46 2.49
 2.52 2.55 2.58 2.61 2.64 2.67 2.7  2.73 2.76 2.79 2.82 2.85 2.88 2.91
 2.94 2.97 3.  ]
[ 4.3240014   3.68037588  2.99121844  2.29146828  1.61635237  0.99930257
  0.46998229  0.05253953 -0.2358098  -0.38578464 -0.396568   -0.27583155
 -0.03925411  0.29043712  0.68484567  1.11162319  1.5364232   1.92497493
  2.24515415  2.46892766  2.57405594  2.5454523   2.37611768  2.06759474
  1.62991409  1.08103483  0.44581216 -0.24544741 -0.95875816 -1.6584

A visualization of the optimized controls and FoM evolution during optimization can be found in the Qudi GUI.